<a href="https://colab.research.google.com/github/LuuZwide/Prioritized-Experience-Replay/blob/main/Project_Prioritized_Replay_ft_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U polygon-api-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.4
    Uninstalling urllib3-2.0.4:
      Successfully uninstalled urllib3-2.0.4


In [ ]:
!pip install keras-tuner -q

In [ ]:
import tensorflow as tf
from google.colab import drive
import pandas as pd
import numpy as np
import math
drive.mount('/content/drive')
import random
from tensorflow import keras
from tensorflow.keras import Model
import math
from datetime import datetime, timedelta
from IPython.core.display import clear_output
import polygon
from polygon import RESTClient
import matplotlib.pyplot as plt
import keras_tuner

KeyboardInterrupt: ignored

In [ ]:
n = 7
today = datetime.today()
new_date = today - timedelta(days=n)
formatted_date = new_date.strftime('%Y-%m-%d')
print(formatted_date)

sysdate = today.strftime('%Y-%m-%d')
pdate = today - timedelta(days=n)
pdate = pdate.strftime('%Y-%m-%d')

print('Candlestict date' ,sysdate)
print('indicators date',pdate )

In [ ]:
forex_client = RESTClient('Y_MQfuWV7a5mqIVZeIiB4Y7P4Z9FrpRq')

In [ ]:
mv_response = forex_client.get_sma(ticker = 'C:EURUSD',timespan = 'minute', window = 7, order = 'desc',timestamp_gt= '2023-08-21', limit =5000 )

ema_response  = forex_client.get_ema(ticker = 'C:EURUSD',timespan = 'minute', window = 7, order = 'desc',timestamp_gt= '2023-08-21', limit =5000)

macd_response = forex_client.get_macd(ticker = 'C:EURUSD',timespan = 'minute',short_window = 12, long_window = 26, signal_window = 9 ,
                                      timestamp_gt= '2023-08-21', series_type = 'close', order = 'desc', limit = 5000)


In [ ]:
def convertToDataFrame(mvIndater,valueName):
    ma_list = []
    for indicator_value in mvIndater.values:
        ma_list.append({
            'timestamp': indicator_value.timestamp,
             'value': indicator_value.value,
         })

    ma_df = pd.DataFrame(ma_list)
    #For debugging purposes
    ma_df = ma_df.rename(columns={'value':valueName})
    return ma_df


def convertToDataFrameMACD(macdIndicator):
    macd_list = []
    for indicator_value in macdIndicator.values:
        macd_list.append({
            'timestamp': indicator_value.timestamp,
            'value':indicator_value.value,
            'signal':indicator_value.signal,
            'histogram':indicator_value.histogram,
        })

    macd_df = pd.DataFrame(macd_list)
    #macd = macd_df[['value','signal','histogram']]
    return macd_df

def generateCandleSticks(client, start_date, end_date, limit = 100 ):

    response =  client.get_aggs(ticker = 'C:EURUSD',multiplier = 1,timespan = 'minute',
                                  from_= start_date, to = end_date, sort = 'desc',limit = limit)

    df = pd.DataFrame(response)
    df['date'] = pd.to_datetime(df['timestamp'], unit='ms')
    final_df = df[[ 'open', 'high', 'low', 'close']]
    final_df = final_df.rename(columns={'open':'Open', 'high':'High', 'low':'Low', 'close':'Close'})

    return final_df,df


def createChart(candleSticks, ma, ema, macd):

    obj = [candleSticks, ma, ema, macd]
    chart = pd.concat(obj, axis = 1)
    chart = chart.dropna()
    return chart

In [ ]:
candleStick, df = generateCandleSticks(forex_client, start_date = pdate, end_date = sysdate, limit = 5000 )
#candleStick.head()
df.head()

In [ ]:
sma = convertToDataFrame(mv_response,'SMA')
sma['date'] = pd.to_datetime(sma['timestamp'], unit='ms')
sma.head()

In [ ]:
ema = convertToDataFrame(mv_response,'EMA')
ema['date'] = pd.to_datetime(ema['timestamp'], unit='ms')
ema.head()

In [ ]:
macd = convertToDataFrameMACD(macd_response)
macd['date'] = pd.to_datetime(macd['timestamp'], unit='ms')
macd.head()

In [ ]:
chart = createChart(candleSticks =candleStick , ma = sma['SMA'], ema = ema['EMA'], macd = macd[['value',	'signal',	'histogram']])
chart.head(10)

In [ ]:
rnnChart = chart.values
rows,cols = rnnChart.shape
print(rows,cols)

def convertToSequence(chart,timeSteps,closes):
  rnnChart = chart.values
  input_sequences = []
  target_sequences = []

  for i in range(len(rnnChart) - timeSteps):
    input_sequences.append(rnnChart[i:i+timeSteps])
    target_sequences.append(closes[i + timeSteps - 1])

  input= np.array(input_sequences)
  target= np.array(target_sequences)

  return input, target

In [ ]:
state = np.random.random((4500, 5, 7))
action = np.random.random((4500, 2))

print("State shape:", state.shape)
print("Action shape:", action.shape)

# Expand dimensions of action to match the time steps dimension of state
action_expanded = np.expand_dims(action, axis=1)
action_expanded = np.tile(action_expanded, (1, 5, 1))  # Repeat along the time steps

# Concatenate state and expanded action along the last axis (feature axis)
combined_array = np.concatenate((state, action_expanded), axis=-1)

print("Combined array shape:", combined_array.shape)

In [ ]:
class Portfolio():
    def __init__(self, starting_amount):
        self.port_value = starting_amount
        self.prev_port_value = starting_amount
        self.starting_amount = starting_amount
        self.transactionCost = starting_amount * 0.003
        self.relative_vector = None
        self.prev_relative_vector = np.array([0, 0])
        self.prev_weights = np.array([0, 0])
        self.hold_weight = 0.0005
        self.returns = []
        self.counter = 0

    def update_port(self, weights, prev_prices, cur_prices):
        prev = np.hstack(([1], prev_prices))
        curr = np.hstack(([1], cur_prices))
        self.relative_vector = curr / prev

        self.prev_weights = weights

        return

    def calculate_reward(self, weights):

        rateReturn = np.dot(self.relative_vector, np.array((0,1)))

        if(np.array_equal(weights, self.prev_weights) == False):
           rateReturn -= 0.000015 #Transaction cost

        rateReturn = rateReturn
        if (weights[1] > 0.9):
          rateReturn = rateReturn - 1.00002
        else:
          rateReturn = -0.00002

        self.counter += 1
        reward = rateReturn
        self.prev_weights = weights
        return reward


    def clear_returns(self):
        self.returns = []
        self.counter = 0

In [ ]:
class Env():
  def __init__(self, Portfolio, chart , closing_amounts, episode_length, last_index ): #add a model to my parameters
    self.chart = chart
    self.last_index = last_index
    self.closing_amounts = closing_amounts
    self.portfolio = Portfolio
    self.og_episode_length = episode_length
    self.episode_length = episode_length
    self.index = 0
    self.counter = 0 # Count the number of days the agent have been trading
    self.done = False
    self.trunc = False
    self.starting_amount = Portfolio.starting_amount
    self.trade_count = 0 # Number of times the agent has traded
    self.prev_porfolio_value = self.portfolio.port_value
    self.current_portfolio_value = self.portfolio.port_value
    self.returns = 0

  def reset(self): # Index is to deside when to start the trading period
    self.episode_length = self.og_episode_length
    self.portfolio = Portfolio(self.starting_amount)
    self.index = random.randint(0, len(self.chart) -  self.last_index )
    initial_weights = np.array([1, 0])
    self.counter = 0
    self.portfolio.prev_weights = [0,0]
    self.portfolio.clear_returns()
    self.state = self.add_gaussian_noise(self.chart[self.index])
    return self.state

  def validation_reset(self):
    self.index = len(chart) - self.last_index #example value = 1000
    initial_weights = np.array([1, 0])
    self.episode_length = 900
    self.counter = 0
    self.portfolio.prev_weights = [0,0]
    self.portfolio.clear_returns()
    self.state = self.add_gaussian_noise(self.chart[self.index])
    return self.state

  def add_gaussian_noise(self, state, mean = 0.0, std_dev = 0.00001):
    noise = np.random.normal(mean, std_dev, size=state.shape)
    noisy_state = state + noise
    return noisy_state

  def check_done(self, portfolio_value):
    done = False
    threshhold =  portfolio_value/self.starting_amount
    if threshhold > 0.01:
      done = False
    else:
      done = True
    return done

  def check_trunc(self):
    if(self.counter > self.episode_length):
      return True
    else:
      return False

  def step(self, action ):

    if ((self.index + 1) < len(self.chart)):
      prev_closing = self.closing_amounts[self.index]
      cur_closing = self.closing_amounts[self.index + 1]
      self.prev_prev_porfolio_value = self.current_portfolio_value
      self.portfolio.update_port(action, prev_closing, cur_closing)
      reward = self.portfolio.calculate_reward( action)
      next_state = self.chart[self.index]
      done = False
      trunc = False
    else:
      next_state = self.chart[self.index]
      done = True
      reward = 0
      trunc = True

    if(self.check_done(self.current_portfolio_value)) is True:
      done = True
      next_state = self.chart[self.index]
      reward = reward
      trunc = False

    if(self.check_trunc() is True):
      trunc = True
      done = False
      reward = reward
      next_state = self.chart[self.index]


    self.counter = self.counter + 1
    self.next_state =  self.add_gaussian_noise(next_state)
    self.reward = reward
    self.done = done
    self.trunc = trunc
    self.index = self.index + 1

    return self.next_state, self.reward, self.done, self.trunc

In [ ]:
class Preprocessor():
  def __init__(self, timesteps, features):
    self.normLayer = keras.layers.Normalization(axis = -1)

  def adaptNorm(self,x):
    self.normLayer.adapt(x)

  def getPreprocessor(self):
    return  self.normLayer

In [ ]:
class PreprocessingModel(tf.keras.Model):
    def __init__(self, timesteps, features):
        super(PreprocessingModel, self).__init__()

        self.input_layer = keras.layers.InputLayer(input_shape=(timesteps, features))
        self.normalisation_layer = keras.layers.Normalization(axis=-1)

    def call(self, state):
        x = self.input_layer(state)
        out = self.normalisation_layer(x)
        return out

In [ ]:
class Actor():
  def __init__(self, timesteps, features, preprocessor, learningRate):
    self.timesteps = timesteps
    self.features = features
    self.preprocessor = preprocessor
    self.learningRate = learningRate

  def lossFunction(self, y_true, y_pred = None):
    loss = -1 * tf.reduce_mean(y_true)
    return loss

  def buildModel(self):
    model = keras.Sequential(name = "Actor_Model")
    model.add(keras.layers.InputLayer(input_shape=(self.timesteps, self.features)))
    model.add(self.preprocessor.getPreprocessor())
    model.add(keras.layers.LSTM(units=256, activation='tanh', return_sequences=True))
    model.add(keras.layers.LSTM(units=256, activation='tanh', return_sequences=False))
    model.add(keras.layers.Dense(2, activation='softmax'))

    model.compile( optimizer = keras.optimizers.Adam(learning_rate = self.learningRate))

    return model

In [ ]:
class Critic():
  def __init__(self, timesteps, features, learningRate):
    self.timesteps = timesteps
    self.features = features
    self.learningRate = learningRate

  def buildModel(self):
    model = keras.Sequential(name = "Critic_Model")
    model.add(keras.layers.InputLayer(input_shape=(self.timesteps, self.features + 2)))
    model.add(keras.layers.LSTM(units=256, activation='tanh', return_sequences=True) )
    model.add(keras.layers.LSTM(units=256, activation='tanh', return_sequences=True))
    model.add(keras.layers.LSTM(units=128, activation='tanh', return_sequences=True))
    model.add(keras.layers.LSTM(units=64, activation='tanh', return_sequences=False))
    model.add(keras.layers.Dense(1))

    model.compile( optimizer = keras.optimizers.Adam(learning_rate = self.learningRate))

    return model

In [ ]:
class ReplayBuffer():
  def __init__(self, max_size, timesteps,features, num_actions):
    self.max_size = max_size
    self.timesteps = timesteps
    self.features = features
    self.num_actions = num_actions
    self.state_memory = np.zeros((self.max_size, timesteps, features))
    self.action_memory = np.zeros((self.max_size, num_actions))
    self.reward_memory = np.zeros(self.max_size)
    self.next_state_memory = np.zeros((self.max_size,  timesteps,features))
    self.terminal_memory= np.zeros(self.max_size, dtype = np.float32)
    self.priorities = np.zeros(self.max_size, dtype = np.float32)
    self.mem_counter = 1

  def store_transitions(self,state, action, reward, done):
    index = self.mem_counter % self.max_size
    self.state_memory[index] = state
    self.action_memory[index] = action
    self.reward_memory[index] = reward
    self.terminal_memory[index] = 1 - int(done) #False = 0 , True = 1 Cause if the state is done G will 0 cause the are no future rewards

    if(max(self.priorities == 0)):
      self.priorities[index] = 1
    else:
      self.priorities[index] = max(self.priorities)

    if self.mem_counter > 0 :
      self.next_state_memory[index - 1] = state
    self.mem_counter += 1

  def GetProbabilities(self, scale_a):
    scaledProbs = self.priorities ** scale_a
    sampleProbs = scaledProbs / sum(scaledProbs)
    return sampleProbs

  def GetImportance(self, probabilities):
    importance = (1 / self.max_size) * (1 / probabilities)
    normImportance = importance / max(importance)
    return normImportance


  def PrioritySample(self,batchSize,scale_a = 1):
    mem_size = min(self.mem_counter, self.max_size)
    sampleProbs = self.GetProbabilities(scale_a)
    batch = random.choices(range(mem_size), k = batchSize , weights = sampleProbs[:mem_size])
    importances = self.GetImportance(sampleProbs[batch])
    states = self.state_memory[batch]
    actions = self.action_memory[batch]
    rewards = self.reward_memory[batch]
    next_states = self.next_state_memory[batch]
    dones = self.terminal_memory[batch]
    indices = batch
    return states, actions, rewards, next_states, dones, importances, indices

  def SetPriorities(self, indices, errors, offset = 0.1):
    for i, e in zip(indices, errors):
      self.priorities[i] = e + offset

  def random_sample(self, batch_size):
    mem_size = min(self.mem_counter, self.max_size)
    batch = np.random.choice(mem_size, batch_size)

    states = self.state_memory[batch]
    actions = self.action_memory[batch]
    rewards = self.reward_memory[batch]
    next_states = self.next_state_memory[batch]
    dones = self.terminal_memory[batch]
    return states, actions, rewards, next_states,dones


  def clear_memory(self):
    self.state_memory = np.zeros((self.max_size, self.input_shape))
    self.action_memory = np.zeros((self.max_size, self.num_actions))
    self.reward_memory = np.zeros(self.max_size)
    self.next_state_memory = np.zeros((self.max_size, self.input_shape))
    self.terminal_memory= np.zeros(self.max_size, dtype = np.float32)
    self.mem_counter = 1


In [ ]:
class Agent():
  def __init__(self, memory , actor_model,critic_model , preprocessor,  batch_size, save_location ,gamma = 0,num_batches = 1):
    self.num_batches = num_batches
    self.features = features
    self.timesteps = timesteps
    self.batch_size = batch_size

    self.Actor = actor_model
    self.Critic = critic_model

    self.memory = memory
    self.learn_counter  = 0
    self.preprocessor = preprocessor

    self.save_location = save_location
    self.act_loc = self.save_location + 'actor'
    self.cri_loc = self.save_location + 'critic'

    self.scale_b = 0.4
    self.scale_a = 0.6


  def choose_action(self, state):
    weights = self.Actor(state)
    action = weights.numpy()[0]
    return action

  def update_memory(self,state, action, reward, done):
    self.memory.store_transitions(state, action , reward, done)

  def save_models(self):
    print('...saving models...')
    self.Actor.save(self.act_loc)
    self.Critic.save(self.cri_loc)

  def load_models(self):
    print('...load model....')
    self.Actor = keras.models.load_model(self.act_loc)
    self.Critic = keras.models.load_model(self.cri_loc)


  def learn(self, step):

    if (self.memory.mem_counter < self.batch_size ) and  (step % self.batch_size != 0)  :
      return 0, 0
    else:
      self.learn_counter += 1
      states, actions, rewards, next_states, dones,importanceWeights ,indices= self.memory.PrioritySample(self.batch_size)
      actions = tf.cast(actions, dtype=tf.float32)

      pstates = self.preprocessor(states)

      criticActions = tf.expand_dims(actions, axis=1)
      criticActions = tf.tile(criticActions, multiples=[1, self.timesteps, 1])
      criticStates = tf.concat([pstates, criticActions], axis=-1)

      with tf.GradientTape() as tape:

        y = tf.cast(tf.expand_dims(rewards, axis=-1), dtype=tf.float32)
        value_func = self.Critic(criticStates)
        critic_loss = tf.losses.mean_squared_error(y, value_func)
        error =  critic_loss
        w = tf.cast(importanceWeights ** self.scale_b, dtype=tf.float32)
        critic_loss = tf.multiply(error, w)
        critic_loss = tf.reduce_mean(critic_loss)

      critic_gradients = tape.gradient(critic_loss, self.Critic.trainable_variables)
      self.Critic.optimizer.apply_gradients(zip(critic_gradients, self.Critic.trainable_variables))

      with tf.GradientTape() as tape:
        current_actions = self.Actor(states)
        current_actions = tf.expand_dims(current_actions, axis=1)
        current_actions = tf.tile(current_actions, multiples=[1, self.timesteps, 1])
        targetActorStates = tf.concat([pstates, current_actions], axis=-1)

        actor_loss = self.Critic(targetActorStates)
        actor_loss = -1 * tf.reduce_mean(actor_loss)

      actor_gradients = tape.gradient(actor_loss, self.Actor.trainable_variables)
      self.Actor.optimizer.apply_gradients(zip(actor_gradients, self.Actor.trainable_variables))

      self.memory.SetPriorities(indices, error)
      self.scale_b = self.scale_b * 1.001

      self.learn_counter = 0
    return actor_loss, critic_loss

In [ ]:
#Hyperparameters
max_size = 20000
learning_rate = 0.0000125
timesteps = 40
critic_lr  =0.00005
episode_length = 120
starting_amount = 200
batch_size = 64
num_episodes = 500

#Data
Xchart, ychart = convertToSequence(chart,timesteps,chart['Close'])
_, timesteps, features = Xchart.shape

#Portfolio
portfolio =  Portfolio(starting_amount = starting_amount)

#Env
env = Env(portfolio, Xchart, ychart , episode_length = episode_length, last_index = 1000)

#Memory
replayBuffer = ReplayBuffer( max_size = max_size, timesteps = timesteps ,features = features,num_actions = 2)

#Preprocessor
Critic_preprocessor = PreprocessingModel(timesteps = timesteps ,features = features)

#Actor Preprocessor
preprocessor = Preprocessor(timesteps = timesteps ,features = features)

#Adapt the preprocessing layers
Critic_preprocessor.normalisation_layer.adapt(Xchart)
preprocessor.adaptNorm(Xchart)


#Models
actor = Actor(timesteps = timesteps, features = features, preprocessor = preprocessor, learningRate = learning_rate)
critic = Critic(timesteps = timesteps, features =features , learningRate = critic_lr)
critic_model = critic.buildModel()
actor_model = actor.buildModel()
actor_model.summary()
critic_model.summary()

#Saved location
save_location = '/content/drive/MyDrive/Models/RNN Models/PRActorCritic/'

#Agent
agent = Agent( memory  = replayBuffer , actor_model = actor_model,critic_model = critic_model , preprocessor = Critic_preprocessor,
              batch_size = batch_size, save_location = save_location)

In [ ]:

class MyHyperModel(keras_tuner.HyperModel):
  def __init__(self, env, actor, number_of_episodes, Replay, preprocessor):
    super(MyHyperModel).__init__()
    self.env = env
    self.actor = actor
    self.num_episodes = number_of_episodes
    self.memory = Replay
    self.preprocessor = preprocessor
    self.timesteps = 40
    self.features = 9

  def returnStates(self):
    states, actions, rewards, _ , dones = self.memory.random_sample(batch_size)
    actions = tf.cast(actions, dtype=tf.float32)
    states = tf.cast(states, dtype=tf.float32)
    rewards = tf.cast(rewards, dtype=tf.float32)
    pstates = self.preprocessor(states)
    criticActions = tf.expand_dims(actions, axis=1)
    criticActions = tf.tile(criticActions, multiples=[1, self.timesteps, 1])
    criticStates = tf.concat([pstates, criticActions], axis=-1)

    return criticStates, states, rewards

  def build(self,hp):
    model = keras.Sequential(name = "Critic_Model")
    model.add(keras.layers.InputLayer(input_shape=(self.timesteps, self.features + 2)))
    model.add(keras.layers.LSTM(units=256, activation='tanh', return_sequences=True) )
    model.add(keras.layers.LSTM(units=256, activation='tanh', return_sequences=True))
    model.add(keras.layers.LSTM(units=128, activation='tanh', return_sequences=True))
    model.add(keras.layers.LSTM(units=64, activation='tanh', return_sequences=False))
    model.add(keras.layers.Dense(1))
    return model

  def fit(self, hp, model, x = None, y = None, validation_data = None, callbacks = None, **kwargs):

    batch_size = 64
    num_batches = 1
    loss_metric = keras.metrics.Mean()
    model.compile(optimizer = keras.optimizers.Adam(hp.Float("learning_rate", max_value = 0.00003, min_value = 0.00001, sampling = "log")))

    @tf.function
    def run_train_steps(criticStates, states, rewards): #must take tensors as inputs
      # for each batch open the gradient tape
      with tf.GradientTape() as tape:
        #run a forward pass
        y = tf.cast(tf.expand_dims(rewards, axis=-1), dtype=tf.float32)
        valueFunction = model(criticStates)
        loss = tf.losses.mean_squared_error(y, valueFunction)
        loss = tf.math.reduce_mean(loss)
      #retrieve the gradients of the model
      gradients = tape.gradient(loss, model.trainable_variables)
      #the is when we apply gradient descent
      model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))

      with tf.GradientTape() as tape:
        current_actions = self.actor(states, training=True)
        current_actions = tf.expand_dims(current_actions, axis=1)
        current_actions = tf.tile(current_actions, multiples=[1, self.timesteps, 1])

        ActorStates = tf.concat([states, current_actions], axis=-1)
        actor_loss = model(ActorStates,  training=True)
        actor_loss = -1 * tf.math.reduce_mean(actor_loss)
      gradients = tape.gradient(actor_loss, self.actor.trainable_variables)
      self.actor.optimizer.apply_gradients(zip(gradients, self.actor.trainable_variables))

    @tf.function
    def run_validation_step(criticStates, states, rewards):
      y = tf.cast(tf.expand_dims(rewards, axis=-1), dtype=tf.float32)
      valueFunction = model(criticStates)
      loss = tf.losses.mean_squared_error(y, valueFunction)
      loss = tf.math.reduce_mean(loss)
      loss_metric.update_state(loss)


    # Run through an episode to get the batches
    step = 0

    for epoch in range(2):
      for e in range(self.num_episodes):
        state = self.env.reset()
        done, trunc = (False, False)
        while not (trunc):
          state = np.expand_dims(state, axis = 0)
          action = agent.choose_action(state)
          step += 1
          next_state, reward, done, trunc = env.step(action)
          self.memory.store_transitions(state, action , reward, done)
          state = next_state

          #Once we have memory big enough or mem_size mod 4 == 0 then we run a training step
          if (self.memory.mem_counter > batch_size ) and (step % 4 == 0):

            for i in range(num_batches):
              criticStates, states, rewards = self.returnStates()
              #Run Training Step
              run_train_steps(criticStates, states, rewards)

      for callback in callbacks:
        callback.model = model

      best_epoch_loss = float("inf")

      # Run validation step (Loss based validation or Reward based validation)
      self.memory.clear_memory()
      state = self.env.validation_reset()
      done, trunc = (False, False)
      while not (trunc):
        state = np.expand_dims(state, axis = 0)
        action = agent.choose_action(state)
        step += 1
        next_state, reward, done, trunc = env.step(action)
        self.memory.store_transitions(state, action , reward, done)
        state = next_state

        #Now we run a validation pass and get the loss
        if (self.memory.mem_counter > batch_size ) and (step % 4 == 0):
          criticStates, states, rewards = self.returnStates()
          run_validation_step(criticStates, states, rewards)

      epoch_loss = float(loss_metric.result().numpy())
      for callback in callbacks:
        callback.on_epoch_end(epoch , logs = {"my_metric": epoch_loss}  )
      loss_metric.reset_states()

      print(f"Epoch loss: {epoch_loss}")
      best_epoch_loss = min(best_epoch_loss, epoch_loss)

    return best_epoch_loss


In [ ]:
env = Env(portfolio, Xchart, ychart , episode_length = 100, last_index = 1000)

model = MyHyperModel(env = env, actor = actor_model, number_of_episodes = 10, Replay = replayBuffer , preprocessor = Critic_preprocessor)


tuner = keras_tuner.RandomSearch(
    hypermodel= model,
    objective= keras_tuner.Objective("my_metric","min"),
)

tuner.search()

In [ ]:
avg_scores = []
avg_rewards = []
avg_reward_history = []
scores = 0
rewards = 0
score_history = []
reward_history = []
actor_loss_history = []
critic_loss_history = []
actor_avg_lesses = []
critic_avg_lesses = []
high_score = -1
step = 0

In [ ]:

for e in range(num_episodes):
    weights_history = []
    highest = 0
    state = env.reset()
    rewards = 0
    done, trunc = (False, False)
    starting_index = env.index
    print('episode: ', e)
    while not (trunc):
        state = np.expand_dims(state, axis = 0)
        action = agent.choose_action(state)
        next_state, reward, done, trunc = env.step(action)
        step += 1
        rewards += reward
        agent.update_memory(state, action, reward, done)
        actor_loss,critic_loss  =  agent.learn(step)
        state = next_state
        weights_history.append(action)
        if (env.current_portfolio_value > highest):
          highest = env.current_portfolio_value

    avg_reward_history.append(rewards)
    avg_reward = np.mean(avg_reward_history[-10:])
    avg_rewards.append(avg_reward)


    if (avg_reward > high_score) & (e > 10) :
      high_score = avg_reward
      agent.save_models()


    if e > 1 :
      actor_loss_history.append(actor_loss)
      avg_loss = np.mean(actor_loss_history)
      actor_avg_lesses.append(avg_loss)

      critic_loss_history.append(critic_loss)
      avg_loss = np.mean(critic_loss_history)
      critic_avg_lesses.append(avg_loss)

    if  e > 10:
        clear_output(wait=True)
        fig, ax = plt.subplots(nrows=5, ncols=1, figsize=(7, 16))
        ax[0].plot(avg_rewards)
        ax[0].set_xlabel('Episodes')
        ax[0].set_ylabel('Average Reward')

        ax[1].plot(actor_avg_lesses)
        ax[1].set_xlabel('Episodes')
        ax[1].set_ylabel('Actor Avg Loss')

        ax[2].plot(critic_avg_lesses)
        ax[2].set_xlabel('Episodes')
        ax[2].set_ylabel('Critic Avg Loss')

        ax[3].plot(ychart[starting_index:env.index])
        ax[3].set_xlabel('Steps')
        ax[3].set_ylabel('Asset Value')

        ax[4].plot(weights_history)
        ax[4].set_xlabel('Steps')
        ax[4].set_ylabel('Weight')
        ax[4].legend(['Cash', 'Asset'])

        plt.show()

In [ ]:
agent.save_models()

In [ ]:
class Test_env()


In [ ]:
#practice run